In [1]:
import dataAck
import portfolio
import time
import hashlib
import sys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
##GET PREDICTIONS OF ALL MODELS IN A PORTFOLIO

allPortfolios = portfolio.getPortfolios()
allPortfolios

[{'description': 'XLY 08-11-17',
  'key': '8d2f03a0255fb23c126417c182491b9c7d5d8181e30e4f51b67022ae'}]

In [3]:
allModels = []
modelsInPortfolio = {}
for portfolioInfo in allPortfolios:
    print(portfolioInfo)
    models = portfolio.getModelsByKey(portfolio.getPortfolioModels(portfolioInfo["key"]))
    modelsInPortfolio[portfolioInfo["key"]] = models
    allModels += models
    for model in models:
        print(model.describe())
    

{'key': '8d2f03a0255fb23c126417c182491b9c7d5d8181e30e4f51b67022ae', 'description': 'XLY 08-11-17'}
((10, ('EWT', 33, None, None), 5, 'XLY'), 10, 250)
((10, ('EWT', 33, None, None), 3, 'XLY'), 10, 50)


In [4]:
tickersRequired = []
tickersTraded = []
for mod in allModels:
    print(mod.describe())
    if mod.inputSeries.targetTicker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.targetTicker)
    if mod.inputSeries.series.ticker not in tickersRequired:
        tickersRequired.append(mod.inputSeries.series.ticker)
    if mod.inputSeries.targetTicker not in tickersTraded:
        tickersTraded.append(mod.inputSeries.targetTicker)
    
pulledData, validTickers = dataAck.downloadTickerData(tickersRequired)

joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])

((10, ('EWT', 33, None, None), 5, 'XLY'), 10, 250)
((10, ('EWT', 33, None, None), 3, 'XLY'), 10, 50)
EWT
XLY


In [2]:
from google.cloud import datastore, storage, logging
import time
import params

In [6]:
##GENERATE AGGREGATE RETURNS FOR ALL MODELS
aggregateReturns = portfolio.generateAggregateReturns(allModels, joinedData)

((10, ('EWT', 33, None, None), 5, 'XLY'), 10, 250)
((10, ('EWT', 33, None, None), 3, 'XLY'), 10, 50)


In [7]:
def storePortfolioAllocation(portfolioKey, lastDataDayUsed, algorithmWeights, tickerAllocation):
    toUpload = {}
    toUpload["portfolio"] = portfolioKey
    toUpload["lastDataDayUsed"] = lastDataDayUsed
    
    for item in algorithmWeights:
        print(item, algorithmWeights[item])
        toUpload["algo_" + item] = algorithmWeights[item]
    
    for item in tickerAllocation:
        toUpload["ticker_" + item] = tickerAllocation[item]
    
    ##UPLOAD ORGANISM OBJECT
    while True:
        try:
            datastoreClient = datastore.Client('money-maker-1236')
            #HASH DIGEST
            predictionHash = hashlib.sha224((str(portfolioKey) + " " + str(toUpload["lastDataDayUsed"])).encode('utf-8')).hexdigest()
            key = datastoreClient.key(params.portfolioAllocation, predictionHash) #NEED TO HASH TO ENSURE NON-OVERLAPPING PREDICTIONS
            organismToStore = datastore.Entity(key=key)
            organismToStore.update(toUpload)
            datastoreClient.put(organismToStore)
            break
        except:
            print("UPLOAD ERROR:", str(sys.exc_info()))
            time.sleep(10)

In [ ]:
def produceHRPPredictions(aggregateReturns, windowSize, maxWindowSize = False):
    hrpReturns = pd.DataFrame([])
    historicalWeights = pd.DataFrame([])
    i = windowSize
    while i < len(aggregateReturns):
        corr = None
        cov = None
        if maxWindowSize == False:
            corr = (aggregateReturns[:i]).corr()
            cov = (aggregateReturns[:i]).cov()
        else:
            corr = (aggregateReturns[i-windowSize:i]).corr()
            cov = (aggregateReturns[i-windowSize:i]).cov()
        weights = hrp.getHRP(cov, corr)
    #     display(weights)
    #     display(aggregateReturns[i+windowSize:i+windowSize+1])
        todayReturn = aggregateReturns[i:i+1] * weights
    #     display(todayReturn)
        sumReturn = pd.DataFrame(todayReturn.apply(lambda x:sum(x), axis=1))
        hrpReturns = pd.concat([hrpReturns, sumReturn])
        i += 1
    return hrpReturns, weights

In [ ]:
##GET AGGREGATE PREDICTION FOR A MODEL
#portfolio.storeAggregateModelPrediction(thisModel, thisDF.values[j][0], thisDF.index[j], shouldReturn=True)


In [8]:
##GENERATE WEIGHTS FOR PORTFOLIO
for portfolioKey in modelsInPortfolio:
    hrpReturns, weights = produceHRPPredictions(aggregateReturns[[str(model.describe()) for model in modelsInPortfolio[portfolioKey]]], 22, True)
    print(portfolioKey, weights)
    
    netPosition = {}
    for model in modelsInPortfolio[portfolioKey]:
        if model.inputSeries.targetTicker not in netPosition:
            netPosition[model.inputSeries.targetTicker] = 0.0
        netPosition[model.inputSeries.targetTicker] += weights[str(model.describe())] * portfolio.getAggregatePredictionForModelDaily(model, joinedData)
    print(portfolioKey, netPosition)
    storePortfolioAllocation(portfolioKey, joinedData.index[-1], weights.to_dict(), netPosition)

8d2f03a0255fb23c126417c182491b9c7d5d8181e30e4f51b67022ae ((10, ('EWT', 33, None, None), 3, 'XLY'), 10, 50)     0.707599
((10, ('EWT', 33, None, None), 5, 'XLY'), 10, 250)    0.292401
dtype: float64
((10, ('EWT', 33, None, None), 5, 'XLY'), 10, 250) [-1.0, 1.0, -1.0, -1.0, 1.0] -0.2
((10, ('EWT', 33, None, None), 3, 'XLY'), 10, 50) [1.0, -1.0, -1.0] -0.3333333333333333
8d2f03a0255fb23c126417c182491b9c7d5d8181e30e4f51b67022ae {'XLY': -0.29434651834238768}
((10, ('EWT', 33, None, None), 3, 'XLY'), 10, 50) 0.707598887568
((10, ('EWT', 33, None, None), 5, 'XLY'), 10, 250) 0.292401112432


In [3]:
def getPortfolioAllocations(portfolioKey):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.portfolioAllocation)
            query.add_filter('portfolio', '=', portfolioKey)
            retrievedPredictions = list(query.fetch())
            return retrievedPredictions
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))
    

In [5]:
getPortfolioAllocations("8d2f03a0255fb23c126417c182491b9c7d5d8181e30e4f51b67022ae")

[<Entity('walkforward_portfolio_allocation', '91e4ba0aff9a22a3ebe133b7e6bdeef6e078e3463cc30d956d364073') {'lastDataDayUsed': datetime.datetime(2017, 8, 10, 0, 0, tzinfo=<UTC>), 'ticker_XLY': -0.2943465183423877, "algo_((10, ('EWT', 33, None, None), 3, 'XLY'), 10, 50)": 0.7075988875679076, "algo_((10, ('EWT', 33, None, None), 5, 'XLY'), 10, 250)": 0.29240111243209244, 'portfolio': '8d2f03a0255fb23c126417c182491b9c7d5d8181e30e4f51b67022ae'}>]